Due to the continuous updates of the <a href="https://github.com/yunsuxiaozi/Yunbase">Yunbase</a> GitHub repository, a static version is used <a href="https://www.kaggle.com/code/yunsuxiaozi/yunbase">here</a>.We are using version 3 here.

In [1]:
source_file_path = '/kaggle/input/yunbase/Yunbase/baseline.py'
target_file_path = '/kaggle/working/baseline.py'
with open(source_file_path, 'r', encoding='utf-8') as file:
    content = file.read()
with open(target_file_path, 'w', encoding='utf-8') as file:
    file.write(content)

In [2]:
!pip install -q --requirement /kaggle/input/yunbase/Yunbase/requirements.txt  \
--no-index --find-links file:/kaggle/input/yunbase/

In [3]:
from baseline import Yunbase

#necessary
import polars as pl#similar to pandas, but with better performance when dealing with large datasets.
import pandas as pd#read csv,parquet
import numpy as np#for scientific computation of matrices
#models(lgb,xgb,cat)
from  lightgbm import LGBMRegressor,LGBMClassifier,log_evaluation,early_stopping
from catboost import CatBoostRegressor,CatBoostClassifier
from xgboost import XGBRegressor,XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer#word2vec feature
import warnings#avoid some negligible errors
#The filterwarnings () method is used to set warning filters, which can control the output method and level of warning information.
warnings.filterwarnings('ignore')

import random#provide some function to generate random_seed.
#set random seed,to make sure model can be recurrented.
def seed_everything(seed):
    np.random.seed(seed)#numpy's random seed
    random.seed(seed)#python built-in random seed
seed_everything(seed=2024)

In [4]:
train=pl.read_csv("/kaggle/input/brist1d/train.csv")
train=train.to_pandas()
test=pl.read_csv("/kaggle/input/brist1d/test.csv")
test=test.to_pandas()
print(len(train),len(test))
train.head()

177024 3644


,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,None,NaN,9.6,None,NaN,9.7,None,...,None,None,None,None,None,None,None,None,None,13.4
1,p01_1,p01,06:25:00,None,NaN,9.7,None,NaN,9.2,None,...,None,None,None,None,None,None,None,None,None,12.8
2,p01_2,p01,06:40:00,None,NaN,9.2,None,NaN,8.7,None,...,None,None,None,None,None,None,None,None,None,15.5
3,p01_3,p01,06:55:00,None,NaN,8.7,None,NaN,8.4,None,...,None,None,None,None,None,None,None,None,None,14.8
4,p01_4,p01,07:10:00,None,NaN,8.4,None,NaN,8.1,None,...,None,None,None,None,None,None,None,None,None,12.7


In [5]:
#six float features
feats=['bg','insulin','hr','steps','cals','activity']
ts=['-5:55', '-5:50', '-5:45', '-5:40', '-5:35', '-5:30', '-5:25', '-5:20', '-5:15', '-5:10', '-5:05', '-5:00', '-4:55', '-4:50', '-4:45', '-4:40', '-4:35', '-4:30', '-4:25', '-4:20', '-4:15', '-4:10', '-4:05', '-4:00', '-3:55', '-3:50', '-3:45', '-3:40', '-3:35', '-3:30', '-3:25', '-3:20', '-3:15', '-3:10', '-3:05', '-3:00', '-2:55', '-2:50', '-2:45', '-2:40', '-2:35', '-2:30', '-2:25', '-2:20', '-2:15', '-2:10', '-2:05', '-2:00', '-1:55', '-1:50', '-1:45', '-1:40', '-1:35', '-1:30', '-1:25', '-1:20', '-1:15', '-1:10', '-1:05', '-1:00', '-0:55', '-0:50', '-0:45', '-0:40', '-0:35', '-0:30', '-0:25', '-0:20', '-0:15', '-0:10', '-0:05', '-0:00']
def get_more_data(df,timestep,mode='train'):#mode is train or test
    full_data=[]
    #12step is 1 hour.
    for start in range(0,len(ts)-timestep-12):
        end=start+timestep
        target=end+12
        #[start:end] target
        hours_data=df[['p_num']+[f+ts[i]  for i in range(start,end) for f in feats]+['bg'+ts[target]]]
        hours_data.columns=['p_num']+[f+str(i-start)  for i in range(start,end)for f in feats]+['bg+1:00']
        hours_data=hours_data[~hours_data['bg+1:00'].isna()]
        full_data.append(hours_data)
    if mode=='test':
        hours_data=df[['p_num']+[f+ts[i]  for i in range(len(ts)-timestep,len(ts)) for f in feats]]
        hours_data.columns=['p_num']+[f+str(i-len(ts)+timestep)  for i in range(len(ts)-timestep,len(ts))for f in feats]
        full_data.append(hours_data)
    full_data=pd.concat(full_data).drop_duplicates()
    return full_data

In [6]:
def FE(df,mode='train',timestep=32):#mode='train' or 'test'
    #mean_isnan is 0.98 ,id is useless.
    print("< drop useless columns >")
    df.drop([c for c in df.columns if 'carbs' in c]+['id'],axis=1,inplace=True)

    #nearest bgs' correlation is greater than 0.99.
    print("< fillna >")
    bgs=[c for c in list(df.columns) if (c!='bg+1:00') and ('bg' in c)]
    df[bgs] = df[bgs].fillna(method='ffill', axis=1)
    df[bgs] = df[bgs].fillna(method='bfill', axis=1)

    #long tail distribution
    print("< log1p transform >")
    step_cols=[c for c in list(df.columns) if 'step' in c]
    df[step_cols]=df[step_cols].fillna(50).astype(np.float32)
    df[step_cols]=np.log1p(df[step_cols])

    #transform None to np.nan.
    print("< fix bug >")
    float_cols=[c for c in df.columns  for f in feats[:-1] if f in c]
    df[float_cols]=df[float_cols].astype(np.float32)

    print("< get more data >")
    df=get_more_data(df,timestep=timestep,mode=mode)
    #p_num to int
    df['p_num_int']=df['p_num'].apply(lambda x:int(x[1:])).astype(np.int8)
    #fillna -1
    float_cols=[col+str(ts) for ts in range(timestep) for col in ['bg','insulin','hr','steps','cals']]
    df[float_cols]=df[float_cols].fillna(-1)

    #activity to str then drop them.
    print("< activity to str >")
    activity_cols=[c for c in list(df.columns) if 'activity' in c]
    activity_feats=df[activity_cols].astype(str).replace('None','').values
    df['activity_str']=[' '.join(activity_feats[i]) for i in range(len(activity_feats))]
    df.drop(activity_cols,axis=1,inplace=True)

    print("< Z-score feature >")
    cols=['bg', 'insulin','steps','hr','cals']
    for col in cols:
        df[f'mean_pnum_{col}{timestep-1}']=df.groupby(['p_num'])[col+f"{timestep-1}"].transform('mean')
        df[f'std_pnum_{col}{timestep-1}']=df.groupby(['p_num'])[col+f"{timestep-1}"].transform('std')
        df[f'norm_pnum_{col}{timestep-1}']=(df[f'{col}{timestep-1}']-df[f'mean_pnum_{col}{timestep-1}'])/df[f'std_pnum_{col}{timestep-1}']
    print("< cross feature >")
    cross=['bg', 'insulin','steps','hr','cals']
    for i in range(len(cross)):
        for j in range(i+1,len(cross)):
            for k in [1]:
                df[cross[i]+f"{timestep-k}+"+cross[j]+f"{timestep-k}"]=df[cross[i]+f"{timestep-k}"]+df[cross[j]+f"{timestep-k}"]
                df[cross[i]+f"{timestep-k}-"+cross[j]+f"{timestep-k}"]=df[cross[i]+f"{timestep-k}"]-df[cross[j]+f"{timestep-k}"]
                df[cross[i]+f"{timestep-k}*"+cross[j]+f"{timestep-k}"]=df[cross[i]+f"{timestep-k}"]*df[cross[j]+f"{timestep-k}"]
                df[cross[i]+f"{timestep-k}/"+cross[j]+f"{timestep-k}"]=df[cross[i]+f"{timestep-k}"]/(df[cross[j]+f"{timestep-k}"]+1e-15)
    #strong correlation between nearest columns
    df.drop([f'bg{i}'for i in range(timestep) if i%2!=0],axis=1,inplace=True)
    return df

train=FE(train,mode='train')
test=FE(test,mode='test')
total=pd.concat((train,test))
test=total[total['bg+1:00'].isna()]
train=total[~total['bg+1:00'].isna()]
print(len(train),len(test),len(total))
train.head()

< drop useless columns >
< fillna >
< log1p transform >
< fix bug >
< get more data >
< activity to str >
< Z-score feature >
< cross feature >
< drop useless columns >
< fillna >
< log1p transform >
< fix bug >
< get more data >
< activity to str >
< Z-score feature >
< cross feature >
371370 3644 375014


,p_num,bg0,insulin0,hr0,steps0,cals0,insulin1,hr1,steps1,cals1,...,steps31*hr31,steps31/hr31,steps31+cals31,steps31-cals31,steps31*cals31,steps31/cals31,hr31+cals31,hr31-cals31,hr31*cals31,hr31/cals31
0,p01,9.6,0.0083,-1.0,3.931826,-1.0,0.0083,-1.0,3.931826,-1.0,...,-3.931826,-3.931826,2.931826,4.931826,-3.931826,-3.931826,-2.0,0.0,1.0,1.0
1,p01,9.7,0.0083,-1.0,3.931826,-1.0,0.0083,-1.0,3.931826,-1.0,...,-3.931826,-3.931826,2.931826,4.931826,-3.931826,-3.931826,-2.0,0.0,1.0,1.0
2,p01,9.2,0.0083,-1.0,3.931826,-1.0,0.0083,-1.0,3.931826,-1.0,...,-3.931826,-3.931826,2.931826,4.931826,-3.931826,-3.931826,-2.0,0.0,1.0,1.0
3,p01,8.7,0.0083,-1.0,3.931826,-1.0,0.0083,-1.0,3.931826,-1.0,...,-3.931826,-3.931826,2.931826,4.931826,-3.931826,-3.931826,-2.0,0.0,1.0,1.0
4,p01,8.4,0.0083,-1.0,3.931826,-1.0,0.0083,-1.0,3.931826,-1.0,...,-3.931826,-3.931826,2.931826,4.931826,-3.931826,-3.931826,-2.0,0.0,1.0,1.0


In [7]:
yunbase=Yunbase(num_folds=5,
                  models=[],
                  FE=None,
                  drop_cols=[],
                  seed=2025,
                  objective='regression',
                  metric='rmse',
                  nan_margin=0.95,
                  group_col='p_num',
                  target_col='bg+1:00',
                  device='gpu',
                  early_stop=250,
                  use_high_corr_feat=False,
                  text_cols=['activity_str'],
                  word2vec_models=[(TfidfVectorizer(max_features=150,ngram_range=(2,3)),'activity_str','tfidf'),
                                   (CountVectorizer(max_features=150,ngram_range=(4,5)),'activity_str','wordcnt')
                                  ]
               )
yunbase.fit(train_path_or_file=train)
test_preds=yunbase.predict(test_path_or_file=test,weights=[4,3,4])
yunbase.submit(submission_path='/kaggle/input/brist1d/sample_submission.csv',test_preds=test_preds)

Currently supported metrics:['custom_metric', 'mae', 'rmse', 'mse', 'medae', 'rmsle', 'auc', 'f1_score', 'mcc', 'accuracy', 'logloss']
Currently supported models:['lgb', 'cat', 'xgb']
Currently supported kfolds:['KFold', 'GroupKFold', 'StratifiedKFold', 'StratifiedGroupKFold']
Currently supported objectives:['binary', 'multi_class', 'regression']
fit......
load train data
train.shape:(371364, 203)
Feature Engineer
< text column's feature >
-> for column activity_str word feature
-> for column activity_str sentence feature
-> for column activity_str paragraph feature
< one hot encoder >
< drop high correlation feature >
drop_cols=['bg31+hr31', 'bg31-hr31', 'insulin31+hr31', 'insulin31-hr31', 'steps31+hr31', 'steps31-hr31', 'insulin31+cals31', 'insulin31-cals31', 'word_len_max', 'word_len_first', 'word_len_last', 'word_len_mean', 'word_len_median', 'word_len_sum', 'sent_len_min', 'sent_len_max', 'sent_len_first', 'sent_len_last', 'sent_len_mean', 'sent_len_median', 'sent_len_sum', 'para_

1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 250 rounds
[100]	valid_0's rmse: 2.20989
[200]	valid_0's rmse: 2.19506
[300]	valid_0's rmse: 2.19114
[400]	valid_0's rmse: 2.19101
[500]	valid_0's rmse: 2.18809
[600]	valid_0's rmse: 2.1897
[700]	valid_0's rmse: 2.18973
Early stopping, best iteration is:
[516]	valid_0's rmse: 2.18745
name:lgb,fold:1
< word2vec >
-> for column activity_str tfidf word2vec feature
-> for column activity_str wordcnt word2vec feature
< word2vec >
-> for column activity_str tfidf word2vec feature
-> for column activity_str wordcnt word2vec feature
Training until validation scores don't improve for 250 rounds
[100]	valid_0's rmse: 2.37079
[200]	valid_0's rmse: 2.36274
[300]	valid_0's rmse: 2.36193
[400]	valid_0's rmse: 2.35977
[500]	valid_0's rmse: 2.36354
[600]	valid_0's rmse: 2.36614
Early stopping, best iteration is:
[404]	valid_0's rmse: 2.35913
name:lgb,fold:2
< word2vec >
-> for column activity_str tfidf word2vec feature
-> for column activity_str wordc

In [8]:
yunbase.train.head()

,p_num,bg0,insulin0,hr0,steps0,cals0,insulin1,hr1,steps1,cals1,...,sent_word_count_min_45.0,sent_word_count_min_44.0,sent_word_count_min_47.0,sent_word_count_min_valuecounts,sent_word_count_max_nan,sent_word_count_first_nan,sent_word_count_last_nan,sent_word_count_mean_nan,sent_word_count_median_nan,sent_word_count_sum_nan
0,p01,9.6,0.0083,-1.0,3.931826,-1.0,0.0083,-1.0,3.931826,-1.0,...,0,0,0,NaN,0,0,0,0,0,0
1,p01,9.7,0.0083,-1.0,3.931826,-1.0,0.0083,-1.0,3.931826,-1.0,...,0,0,0,NaN,0,0,0,0,0,0
2,p01,9.2,0.0083,-1.0,3.931826,-1.0,0.0083,-1.0,3.931826,-1.0,...,0,0,0,NaN,0,0,0,0,0,0
3,p01,8.7,0.0083,-1.0,3.931826,-1.0,0.0083,-1.0,3.931826,-1.0,...,0,0,0,NaN,0,0,0,0,0,0
4,p01,8.4,0.0083,-1.0,3.931826,-1.0,0.0083,-1.0,3.931826,-1.0,...,0,0,0,NaN,0,0,0,0,0,0
